In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm


/var/folders/x1/cmq6ht1x7ygffd5zpgqj2kv80000gn/T/ipykernel_2326/3509410640.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
Path = '/Users/chenjianfeng/Desktop/Hackathon/Trustworthy AI Hacathon/data/train/'
ads = pd.read_csv(Path + 'train_data_ads.csv') 
feeds = pd.read_csv(Path + 'train_data_feeds.csv')

# Advertisers 

### Advertisers (providing `ads.csv`): They concentrate on user interactions with advertisements.
- log_id
- label
- user_id
- age
- gender
- residence
- city
- city_rank
- series_dev
- series_group
- emui_dev
- device_name
- device_size
- net_type
- task_id
- adv_id
- creat_type_cd
- adv_prim_id
- inter_type_cd
- slot_id
- site_id
- spread_app_id
- hispace_app_tags
- app_second_class
- app_score
- ad_click_list_v001
- ad_click_list_v002
- ad_click_list_v003
- ad_close_list_v001
- ad_close_list_v002
- ad_close_list_v003
- pt_d
- u_newsCatInterestsST
- u_refreshTimes
- u_feedLifeCycle

In [3]:
ads = ads.head(100)

In [4]:
for column in ads.columns:
    print(f"- {column}")

- log_id
- label
- user_id
- age
- gender
- residence
- city
- city_rank
- series_dev
- series_group
- emui_dev
- device_name
- device_size
- net_type
- task_id
- adv_id
- creat_type_cd
- adv_prim_id
- inter_type_cd
- slot_id
- site_id
- spread_app_id
- hispace_app_tags
- app_second_class
- app_score
- ad_click_list_v001
- ad_click_list_v002
- ad_click_list_v003
- ad_close_list_v001
- ad_close_list_v002
- ad_close_list_v003
- pt_d
- u_newsCatInterestsST
- u_refreshTimes
- u_feedLifeCycle


# Publishers 

#### Publishers (providing `feeds.csv`): They focus on user interactions with news content.

- u_userId
- u_phonePrice
- u_browserLifeCycle
- u_browserMode
- u_feedLifeCycle
- u_refreshTimes
- u_newsCatInterests
- u_newsCatDislike
- u_newsCatInterestsST
- u_click_ca2_news
- i_docId
- i_s_sourceId
- i_regionEntity
- i_cat
- i_entities
- i_dislikeTimes
- i_upTimes
- i_dtype
- e_ch
- e_m
- e_po
- e_pl
- e_rn
- e_section
- e_et
- label
- cillabel
- pro

In [5]:
feeds = feeds.head(100)

In [6]:
for column in feeds.columns:
    print(f"- {column}")

- u_userId
- u_phonePrice
- u_browserLifeCycle
- u_browserMode
- u_feedLifeCycle
- u_refreshTimes
- u_newsCatInterests
- u_newsCatDislike
- u_newsCatInterestsST
- u_click_ca2_news
- i_docId
- i_s_sourceId
- i_regionEntity
- i_cat
- i_entities
- i_dislikeTimes
- i_upTimes
- i_dtype
- e_ch
- e_m
- e_po
- e_pl
- e_rn
- e_section
- e_et
- label
- cillabel
- pro


## Data Merging

In [7]:
ads_columns = set(ads.columns)
feeds_columns = set(feeds.columns)
common_columns = ads_columns.intersection(feeds_columns)

if len(common_columns) > 0:
    print("The 'ads.csv' and 'feeds.csv' files have the following common columns:")
    for column in common_columns:
        print(column)
else:
    print("The 'ads.csv' and 'feeds.csv' files do not have any common columns.")

The 'ads.csv' and 'feeds.csv' files have the following common columns:
label
u_newsCatInterestsST
u_refreshTimes
u_feedLifeCycle


In [8]:
distinct_user_ads = ads['user_id'].nunique()
print("Number of distinct user IDs:", distinct_user_ads)

Number of distinct user IDs: 7


In [9]:
distinct_user_feeds = feeds['u_userId'].nunique()
print("Number of distinct user IDs:", distinct_user_feeds)

Number of distinct user IDs: 14


In [10]:
ads_user_ids = set(ads['user_id'])
feeds_user_ids = set(feeds['u_userId'])

if ads_user_ids.intersection(feeds_user_ids):
    print("ads and feeds have", len(ads_user_ids.intersection(feeds_user_ids)), "common user_ids.")

In [11]:
for column in ads.columns:
    if ads[column].dtype == object: #int or float
        print(f"Column: {column}, Data Type: {ads[column].dtype}")

Column: ad_click_list_v001, Data Type: object
Column: ad_click_list_v002, Data Type: object
Column: ad_click_list_v003, Data Type: object
Column: ad_close_list_v001, Data Type: object
Column: ad_close_list_v002, Data Type: object
Column: ad_close_list_v003, Data Type: object
Column: u_newsCatInterestsST, Data Type: object


In [12]:
object_columns = ads.select_dtypes(include='object').columns
object_data = ads[object_columns]
object_data.tail(5)

,ad_click_list_v001,ad_click_list_v002,ad_click_list_v003,ad_close_list_v001,ad_close_list_v002,ad_close_list_v003,u_newsCatInterestsST
95,14737^25802^32492^13240^33873,1126^1037^1730^1719^1382,305^327^344^317^301,24107,1218,173,16^173^168^151^65
96,14737^25802^32492^13240^33873,1126^1037^1730^1719^1382,305^327^344^317^301,24107,1218,173,16^173^168^151^65
97,14737^25802^32492^13240^33873,1126^1037^1730^1719^1382,305^327^344^317^301,24107,1218,173,16^173^168^151^65
98,14737^25802^32492^13240^33873,1126^1037^1730^1719^1382,305^327^344^317^301,24107,1218,173,16^173^168^151^65
99,14737^25802^32492^13240^33873,1126^1037^1730^1719^1382,305^327^344^317^301,24107,1218,173,16^173^168^151^65


In [13]:
object_columns = ads.select_dtypes(include='object').columns

# delete the '^' 
for col in object_columns:
    ads[col] = ads[col].apply(lambda x: x.split('^'))

# fixed length
fixed_length = 5

# pad or truncate 
def pad_or_truncate(lst, length):
    if len(lst) > length:
        return lst[:length]
    else:
        return lst + ['0'] * (length - len(lst))

for col in object_columns:
    ads[col] = ads[col].apply(lambda x: pad_or_truncate(x, fixed_length))

In [14]:
ads[object_columns].head()

,ad_click_list_v001,ad_click_list_v002,ad_click_list_v003,ad_close_list_v001,ad_close_list_v002,ad_close_list_v003,u_newsCatInterestsST
0,"[30157, 30648, 14278, 31706, 0]","[2066, 1776, 1036, 0, 0]","[114, 219, 312, 0, 0]","[24107, 0, 0, 0, 0]","[1218, 0, 0, 0, 0]","[173, 0, 0, 0, 0]","[39, 220, 16, 0, 0]"
1,"[30157, 30648, 14278, 31706, 0]","[2066, 1776, 1036, 0, 0]","[114, 219, 312, 0, 0]","[24107, 0, 0, 0, 0]","[1218, 0, 0, 0, 0]","[173, 0, 0, 0, 0]","[39, 220, 16, 0, 0]"
2,"[30157, 30648, 14278, 31706, 0]","[2066, 1776, 1036, 0, 0]","[114, 219, 312, 0, 0]","[24107, 0, 0, 0, 0]","[1218, 0, 0, 0, 0]","[173, 0, 0, 0, 0]","[39, 220, 16, 0, 0]"
3,"[30157, 30648, 14278, 31706, 0]","[2066, 1776, 1036, 0, 0]","[114, 219, 312, 0, 0]","[24107, 0, 0, 0, 0]","[1218, 0, 0, 0, 0]","[173, 0, 0, 0, 0]","[39, 220, 16, 0, 0]"
4,"[30157, 30648, 14278, 31706, 0]","[2066, 1776, 1036, 0, 0]","[114, 219, 312, 0, 0]","[24107, 0, 0, 0, 0]","[1218, 0, 0, 0, 0]","[173, 0, 0, 0, 0]","[39, 220, 16, 0, 0]"


In [15]:
# split into multiple columns
def split_list_column(df, column_name, new_column_prefix):
    max_length = df[column_name].apply(len).max()
    split_columns = pd.DataFrame(df[column_name].tolist(),
                                 columns=[f'{new_column_prefix}_{i+1}' for i in range(max_length)])
    df = df.drop(columns=[column_name]).join(split_columns)
    return df

ads = split_list_column(ads, 'ad_click_list_v001', 'ad_click_list_v001')
ads = split_list_column(ads, 'ad_click_list_v002', 'ad_click_list_v002')
ads = split_list_column(ads, 'ad_click_list_v003', 'ad_click_list_v003')
ads = split_list_column(ads, 'ad_close_list_v001', 'ad_close_list_v001')
ads = split_list_column(ads, 'ad_close_list_v002', 'ad_close_list_v002')
ads = split_list_column(ads, 'ad_close_list_v003', 'ad_close_list_v003')
ads = split_list_column(ads, 'u_newsCatInterestsST', 'u_newsCatInterestsST')

In [16]:
for column in ads.columns:
    if ads[column].dtype == object: #int or float
        print(f"Column: {column}, Data Type: {ads[column].dtype}")

Column: ad_click_list_v001_1, Data Type: object
Column: ad_click_list_v001_2, Data Type: object
Column: ad_click_list_v001_3, Data Type: object
Column: ad_click_list_v001_4, Data Type: object
Column: ad_click_list_v001_5, Data Type: object
Column: ad_click_list_v002_1, Data Type: object
Column: ad_click_list_v002_2, Data Type: object
Column: ad_click_list_v002_3, Data Type: object
Column: ad_click_list_v002_4, Data Type: object
Column: ad_click_list_v002_5, Data Type: object
Column: ad_click_list_v003_1, Data Type: object
Column: ad_click_list_v003_2, Data Type: object
Column: ad_click_list_v003_3, Data Type: object
Column: ad_click_list_v003_4, Data Type: object
Column: ad_click_list_v003_5, Data Type: object
Column: ad_close_list_v001_1, Data Type: object
Column: ad_close_list_v001_2, Data Type: object
Column: ad_close_list_v001_3, Data Type: object
Column: ad_close_list_v001_4, Data Type: object
Column: ad_close_list_v001_5, Data Type: object
Column: ad_close_list_v002_1, Data Type:

In [17]:
new_object_columns = ads.select_dtypes(include='object').columns
ads[new_object_columns].head()

,ad_click_list_v001_1,ad_click_list_v001_2,ad_click_list_v001_3,ad_click_list_v001_4,ad_click_list_v001_5,ad_click_list_v002_1,ad_click_list_v002_2,ad_click_list_v002_3,ad_click_list_v002_4,ad_click_list_v002_5,...,ad_close_list_v003_1,ad_close_list_v003_2,ad_close_list_v003_3,ad_close_list_v003_4,ad_close_list_v003_5,u_newsCatInterestsST_1,u_newsCatInterestsST_2,u_newsCatInterestsST_3,u_newsCatInterestsST_4,u_newsCatInterestsST_5
0,30157,30648,14278,31706,0,2066,1776,1036,0,0,...,173,0,0,0,0,39,220,16,0,0
1,30157,30648,14278,31706,0,2066,1776,1036,0,0,...,173,0,0,0,0,39,220,16,0,0
2,30157,30648,14278,31706,0,2066,1776,1036,0,0,...,173,0,0,0,0,39,220,16,0,0
3,30157,30648,14278,31706,0,2066,1776,1036,0,0,...,173,0,0,0,0,39,220,16,0,0
4,30157,30648,14278,31706,0,2066,1776,1036,0,0,...,173,0,0,0,0,39,220,16,0,0


In [18]:
for column in new_object_columns:
    ads[column] = ads[column].astype(int)

In [19]:
# GAN model architecture
class Generator(nn.Module): 
    def __init__(self, input_size, output_size):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(True),
            nn.Linear(128, 256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, output_size)
        )
    
    def forward(self, x):
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self, input_size):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.model(x)

In [20]:
# hyperparameters
input_size = 100  
output_size = len(ads.columns)  
num_epochs = 10000
batch_size = 128
learning_rate = 0.0002

# initialization
generator = Generator(input_size, output_size)
discriminator = Discriminator(output_size)

# loss function & optimizer
criterion = nn.BCELoss()
generator_optimizer = optim.Adam(generator.parameters(), lr=learning_rate)
discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=learning_rate)

# standardize
scaler = StandardScaler()
ads_scaled = scaler.fit_transform(ads)

# convert to tensor
ads_data = torch.tensor(ads_scaled, dtype=torch.float32)

In [21]:
# Training
for epoch in range(num_epochs):
    
    # clear gradients
    discriminator_optimizer.zero_grad() 
    
    # scale real data
    real_samples = ads_data[torch.randint(0, len(ads_data), (batch_size,))]
    real_labels = torch.ones(batch_size, 1)
    
    # generate fake data
    noise = torch.randn(batch_size, input_size)
    fake_samples = generator(noise)
    fake_labels = torch.zeros(batch_size, 1)
    
    # loss for discriminator
    real_output = discriminator(real_samples)
    fake_output = discriminator(fake_samples.detach())
    real_loss = criterion(real_output, real_labels)
    fake_loss = criterion(fake_output, fake_labels)
    discriminator_loss = real_loss + fake_loss
    
    discriminator_loss.backward()
    discriminator_optimizer.step()

    # clear gradients
    generator_optimizer.zero_grad() 
    
    # generate fake data
    noise = torch.randn(batch_size, input_size)
    fake_samples = generator(noise)
    fake_output = discriminator(fake_samples)
    
    # loss for generator
    generator_loss = criterion(fake_output, real_labels)
    
    generator_loss.backward()
    generator_optimizer.step()
    
    # Print loss and other information every 100 epochs
    if (epoch + 1) % 100 == 0:
        # Calculate discriminator accuracy
        real_accuracy = (real_output > 0.5).float().mean().item()
        fake_accuracy = (fake_output < 0.5).float().mean().item()
        d_accuracy = 0.5 * (real_accuracy + fake_accuracy)

        # Calculate generator accuracy
        g_accuracy = (fake_output > 0.5).float().mean().item()

        # Print losses and accuracies
        tqdm.write(f"Epoch [{epoch + 1}/{num_epochs}], "
                f"Generator Loss: {generator_loss.item():.4f}, "
                f"Discriminator Loss: {discriminator_loss.item():.4f}, "
                f"Discriminator Accuracy: {d_accuracy:.4f}, "
                f"Generator Accuracy: {g_accuracy:.4f}")

Epoch [100/10000], Generator Loss: 9.6420, Discriminator Loss: 0.0695, Discriminator Accuracy: 1.0000, Generator Accuracy: 0.0000
Epoch [200/10000], Generator Loss: 14.6156, Discriminator Loss: 0.0257, Discriminator Accuracy: 1.0000, Generator Accuracy: 0.0000
Epoch [300/10000], Generator Loss: 6.5436, Discriminator Loss: 0.0115, Discriminator Accuracy: 1.0000, Generator Accuracy: 0.0000
Epoch [400/10000], Generator Loss: 8.2769, Discriminator Loss: 0.0144, Discriminator Accuracy: 1.0000, Generator Accuracy: 0.0000
Epoch [500/10000], Generator Loss: 8.2029, Discriminator Loss: 0.0311, Discriminator Accuracy: 1.0000, Generator Accuracy: 0.0000
Epoch [600/10000], Generator Loss: 14.6724, Discriminator Loss: 0.0837, Discriminator Accuracy: 0.9961, Generator Accuracy: 0.0000
Epoch [700/10000], Generator Loss: 7.9497, Discriminator Loss: 0.2490, Discriminator Accuracy: 0.9688, Generator Accuracy: 0.0078
Epoch [800/10000], Generator Loss: 9.1455, Discriminator Loss: 0.0623, Discriminator Acc

In [22]:
# generate new data point
noise = torch.randn(100, input_size)
generated_data = generator(noise).detach().numpy()
generated_data = scaler.inverse_transform(generated_data) # inverse transform
generated_data[generated_data < 0] = 0 # set negative values to 0
generated_data = np.round(generated_data).astype(int)  # round to nearest integer
generated_df = pd.DataFrame(generated_data, columns=ads.columns) # convert to dataframe
generated_df.head()

,log_id,label,user_id,age,gender,residence,city,city_rank,series_dev,series_group,...,ad_close_list_v003_1,ad_close_list_v003_2,ad_close_list_v003_3,ad_close_list_v003_4,ad_close_list_v003_5,u_newsCatInterestsST_1,u_newsCatInterestsST_2,u_newsCatInterestsST_3,u_newsCatInterestsST_4,u_newsCatInterestsST_5
0,473727,0,100023,5,2,26,162,5,34,3,...,188,7,0,0,13,89,74,190,35,5
1,508364,0,100021,5,2,26,175,5,31,3,...,175,0,13,0,0,94,82,146,17,5
2,515699,0,100020,5,2,27,177,5,31,3,...,175,0,16,0,0,92,82,143,19,5
3,495122,0,100021,5,2,25,169,5,32,3,...,178,0,4,0,0,100,78,153,14,5
4,436406,0,100023,4,2,25,162,5,34,3,...,196,24,0,0,23,79,72,190,54,5


In [23]:
ads.head()

,log_id,label,user_id,age,gender,residence,city,city_rank,series_dev,series_group,...,ad_close_list_v003_1,ad_close_list_v003_2,ad_close_list_v003_3,ad_close_list_v003_4,ad_close_list_v003_5,u_newsCatInterestsST_1,u_newsCatInterestsST_2,u_newsCatInterestsST_3,u_newsCatInterestsST_4,u_newsCatInterestsST_5
0,373250,0,100005,3,2,16,147,2,32,6,...,173,0,0,0,0,39,220,16,0,0
1,373253,1,100005,3,2,16,147,2,32,6,...,173,0,0,0,0,39,220,16,0,0
2,373252,1,100005,3,2,16,147,2,32,6,...,173,0,0,0,0,39,220,16,0,0
3,373251,0,100005,3,2,16,147,2,32,6,...,173,0,0,0,0,39,220,16,0,0
4,373255,0,100005,3,2,16,147,2,32,6,...,173,0,0,0,0,39,220,16,0,0
